<a href="https://colab.research.google.com/github/wcrowley342/Donordata/blob/main/Donation_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

text/copy on how nonprofits can benefit from data analysis. Brief intro of unscripted and description of data in order to run through a real life example

fivethirtyeight style guide usage links/resources


*   https://www.dataquest.io/blog/making-538-plots/
*   https://matplotlib.org/stable/gallery/style_sheets/fivethirtyeight.html
*   https://towardsdatascience.com/a-beginners-guide-to-plotting-fivethrityeight-like-visualizations-5b63d3f3ddd0



# Data import & Prep

In [ ]:
#import necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#use FiveThirtyEight Style
import matplotlib.style as style
plt.style.use('fivethirtyeight')

#import data
donations = pd.read_csv('https:[ENTER GITHUB RAW URL HERE')

#preview/inspect data
donations.head()

Use that sweet dataprep package? 
https://medium.com/towards-data-science/speeding-up-exploratory-data-analysis-with-python-838fe5e25b43  

In [ ]:
#check number of rows/columns

print(donations.shape)

In [ ]:
#review columns / summary statistics
donations.describe()

In [ ]:
#.info() function: Explore data types and nulls/ NaNs in your data
donations.info()

In [ ]:
#check null values
donations.isnull().sum

# Data cleanup

In [ ]:
#dropping some columns

donations = donations.drop(columns=['Total'])

donations.head(5)

In [ ]:
#feature engineering / creating columns

donations['Total'] = donations['HP'] + donations['Attack']

# EDA (visual)

In [ ]:
#add in 538 formatting

In [ ]:
#histograms

title ='Medium blogs text length'
x ='Number of charecters in post'
y = 'Frequency'

data['claps'].plot(kind='hist',bins=30)

In [ ]:
#boxplots
data.plot(kind='box',x='reading_time',y='text_len')

In [ ]:
#heatmap based on STATE!! ($$ > COUNT of donations )

# EDA (statistical)

In [ ]:
#average values

donations.mean()

# Comparing Campaigns / Group by

In [ ]:
#group by type, sort by best attack

df.groupby(['Type 1']).mean().sort_values('Attack',ascending=False)

# Segmenting donors

In [ ]:
#RFM analysis???

In [ ]:
#read speicfic rows of donations

donations.loc[donations['Amount'] > 400]

In [ ]:
#heatmap based on STATE!! ($$ > COUNT of donations )